In [ ]:
%%capture
!pip install langchain openai
!pip install -q -U faiss-cpu tiktoken

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Open AI API Key:")

# Introduction

Large Language Models (LLMs) are powerful tools for generating human-like text, but they have limitations.

Retrieval Augmented Generation (RAG) addresses these challenges, enhancing LLMs by integrating retrieval mechanisms. This approach ensures that the content LLMs produce is both contextually relevant and factually accurate. RAG acts as a bridge, connecting LLMs to vast knowledge sources. As AI becomes increasingly used for diverse tasks, the accuracy and relevance of the generated information are crucial.

RAG meets this demand, making AI interactions more informative and context-aware.

# What You Need for RAG Implementation

Before building out a RAG system, it's essential to familiarize yourself with the tools that make this process possible.

Each tool plays a specific role, ensuring that the RAG system operates efficiently and effectively.

**LLM**: At the heart of the system is the LLM, the core AI model responsible for generating human-like text responses.

**Vector Store**: This is where the magic happens. The Vector Store is a dedicated storage system that houses embeddings and their corresponding textual data, ensuring quick and efficient retrieval.

**Vector Store Retriever**: Think of this as the search engine of the system. The Vector Store Retriever fetches relevant documents by comparing vector similarities, ensuring that the most pertinent information is always at hand.

**Embedder**: Before storing or retrieving data, we need to convert textual information into a format the system can understand. The Embedder takes on this role, transforming text into vector representations.

**Prompt**: Every interaction starts with a user's query or statement. The Prompt captures this initial input, setting the stage for the retrieval and generation processes.

**Document Loader**: With vast amounts of data to process, the Document Loader is essential. It imports and reads documents, preparing them for chunking and embedding.

**Document Chunker**: To make the data more manageable and efficient for retrieval, the Document Chunker breaks documents into smaller, more digestible pieces.

**User Input**: Last but not least, the User Input tool captures the query or statement provided by the end-user, initiating the entire RAG process.


# The RAG System and Its Subsystems

The primary goal of RAG is to provide LLMs with contextually relevant and factually accurate information, ensuring that the generated content meets the highest standards of quality and relevance.

To achieve this, the RAG system is divided into subsystems, each playing a crucial role in the overall process. The tools integral to the RAG system are not standalone entities; they interweave to form the subsystems that drive the RAG process.

Each tool fits within one of the following subsystems:

1) Index

2) Retrieval

3) Augment

These work together as an orchestrated flow that transforms a user's query into a contextually rich and accurate response.


# Index System

**Purpose:** This subsystem is responsible for preparing and organizing the data for efficient retrieval.

Here are the steps of the Index system

**1) Load Documents (Document Loader)**: Imports and reads the vast amounts of data that the system will use.

**2) Chunk Documents (Document Chunker):** Breaks down the loaded documents into smaller, more manageable pieces to facilitate efficient retrieval.

**3) Embed Documents (Embedder):** Converts these textual chunks into vector representations, making them searchable within the system.

**4) Store Embeddings (Vector Store):** Safely stores the generated embeddings alongside their textual counterparts for future retrieval.

### Load documents

In [ ]:
from langchain.document_loaders import WebBaseLoader

yolo_nas_loader = WebBaseLoader("https://deci.ai/blog/yolo-nas-object-detection-foundation-model/").load()

decicoder_loader = WebBaseLoader("https://deci.ai/blog/decicoder-efficient-and-accurate-code-generation-llm/#:~:text=DeciCoder's%20unmatched%20throughput%20and%20low,re%20obsessed%20with%20AI%20efficiency.").load()

yolo_newsletter_loader = WebBaseLoader("https://deeplearningdaily.substack.com/p/unleashing-the-power-of-yolo-nas").load()

### Chunk documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len
)

yolo_nas_chunks = text_splitter.transform_documents(yolo_nas_loader)
decicoder_chunks = text_splitter.transform_documents(decicoder_loader)
yolo_newsletter_chunks = text_splitter.transform_documents(yolo_newsletter_loader)

### Create an index


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cachce/")

# create an embedder
core_embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace = core_embeddings_model.model
)

# store embeddings in vector store
vectorstore = FAISS.from_documents(yolo_nas_chunks, embedder)
vectorstore.add_documents(decicoder_chunks)
vectorstore.add_documents(yolo_newsletter_chunks)

# instantiate a retriever
retriever = vectorstore.as_retriever()

# Retrieval System

**Purpose:** As the name suggests, this subsystem fetches the most relevant information based on the user's query.

Here are the steps in the Retrieval system

**1) Obtain User Query (User Input):** Captures the user's question or statement.

**2) Embed User Query (Embedder):** Transforms the user's query into a vector format, similar to the indexed documents.

**3) Vector Search (Vector Store Retriever):** Searches the Vector Store for documents with embeddings that closely match the embedded user query.

**4) Return Relevant Documents:** The system then returns the top matches, ensuring that the most pertinent information is always provided.


In [ ]:
from langchain.llms.openai import OpenAIChat
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

In [ ]:
llm = OpenAIChat()
handler =  StdOutCallbackHandler()

In [ ]:
# this is the entire retrieval system
qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

# Augment System

**Purpose:** This subsystem enhances the LLM's input prompt with the retrieved context, ensuring that the model has all the necessary information to generate a comprehensive response.

**1) Create Initial Prompt (Prompt):** Starts with the original user query or statement.

**2) Augment Prompt with Retrieved Context:** Merges the initial prompt with the context retrieved from the Vector Store, creating an enriched input for the LLM.

**3) Send Augmented Prompt to LLM:** The enhanced prompt is then fed to the LLM.

**4) Receive LLM's Response:** After processing the augmented prompt, the LLM generates and returns its response.


In [ ]:
response = qa_with_sources_chain({"query":"What does Neural Architecture Search have to do with how Deci creates its models?"})

In [ ]:
print(response['result'])

In [ ]:
response = qa_with_sources_chain({"query":"What is DeciCoder"})

In [ ]:
print(response['result'])

In [ ]:
response = qa_with_sources_chain({"query":"Write a blog about Deci and how it used NAS to generate YOLO-NAS and DeciCoder"})

In [ ]:
print(response['result'])